In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from response.feature import Feature
F_path = "../X_y_data/response2/"
F_name = "Classify_F3.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response2/Classify_F3.pickle


In [3]:
model_path = "../models/response2/"
model_name = "Classify_M3.pickle"
modelM = DataManager(model_path)
lr = modelM.load_data(model_name)

success load : ../models/response2/Classify_M3.pickle


In [4]:
path = "../eval_labeled/"
# path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [5]:
def make_Xy(convs, N=4):
    errors = ["Ignore question", 'Ignore offer', 'Ignore proposal', "Ignore greeting"]
    # errors = errors[:1]
    
    X = []
    y = []
    for conv in convs:

        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_exist_error():
                X.append( dialogue[-N:] )
                    # X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
        
    return X, y

In [6]:
N = 2
X_str, y = make_Xy(convs, N=N)

In [7]:
from tqdm import tqdm
print(F.feature_num)
usr = []
for i, u_s in enumerate( tqdm(X_str) ):
    x = F.featurization(u_s[0])
    usr.append(x)
usr_ = np.array(usr)

  0%|          | 1/1386 [00:00<04:03,  5.69it/s]

67906


100%|██████████| 1386/1386 [04:07<00:00,  5.59it/s]


In [8]:
question_pred = lr.predict(usr_)

In [9]:
def vec2feature(vector):
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [10]:
from sentence_transformers import SentenceTransformer
bert_path = "../../corpus/pretrained/sbert_ignore"
sbert = SentenceTransformer(bert_path)

[647] 2022-01-24 11:55:21,349 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_ignore
[647] 2022-01-24 11:55:22,572 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [11]:
X_all_str = sum(X_str, [])
x_length = len(X_all_str)//N
X_vec = sbert.encode(X_all_str).reshape(x_length, N, -1)
X = np.array([ vec2feature(vec) for vec in X_vec ])

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [13]:
class ResponseModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(ResponseModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [14]:
smodel_path = "../models/response2/"
smodel_name = "sbert_ffn.pickle"
smodelM = DataManager(smodel_path)
model = smodelM.load_data(smodel_name)

success load : ../models/response2/sbert_ffn.pickle


In [15]:
with torch.no_grad():
    X_tensor = torch.tensor(X, device='cuda:0').float()
            # 推論
    sbert_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

In [16]:
# count = 0
y_pred = []
for i, p in enumerate(question_pred):
    if p==1:
        if sbert_pred[i] ==1:
            y_pred.append(1)
        else:
            y_pred.append(0)
        # count+=1
    else:
        y_pred.append(0)

In [17]:
score(y, y_pred)

confusion matrix = 
 [[988  44]
 [ 84 270]]
accuracy =  0.9076479076479076
precision =  0.8598726114649682
recall =  0.7627118644067796
f1 score =  0.8083832335329342


In [18]:
score(y, sbert_pred)

confusion matrix = 
 [[984  48]
 [ 70 284]]
accuracy =  0.9148629148629148
precision =  0.8554216867469879
recall =  0.8022598870056498
f1 score =  0.8279883381924199


- 前向き機能 + sbert 

        confusion matrix = 
        [[988  44]
        [ 84 270]]
        accuracy =  0.9076479076479076
        precision =  0.8598726114649682
        recall =  0.7627118644067796
        f1 score =  0.8083832335329342

- sbert のみ

        confusion matrix = 
        [[984  48]
        [ 70 284]]
        accuracy =  0.9148629148629148
        precision =  0.8554216867469879
        recall =  0.8022598870056498
        f1 score =  0.8279883381924199

In [19]:
data_path = "../X_y_data/y_pred/"
data_name = "ignore.pickle"
dataM = DataManager(data_path)
dataM.save_data(data_name, [y, y_pred])

success save : ../X_y_data/y_pred/ignore.pickle
